## Setup

In [1]:
import torch

In [2]:
torch.__version__

'0.4.0'

In [3]:
import pyro

In [4]:
pyro.__version__

'0.2.0'

In [5]:
import os

In [6]:
import numpy as np
import torch.nn as nn

In [7]:
from pyro.distributions import Normal
from pyro.infer import SVI, Trace_ELBO
from pyro.optim import Adam

In [8]:
smoke_test = ('CI' in os.environ)
pyro.enable_validation(True)

In [9]:
os.environ

environ({'PWD': '/home/ivan/environments/my_env1/DL_1', 'INSTANCE': '', 'XDG_GREETER_DATA_DIR': '/var/lib/lightdm-data/ivan', 'XDG_SESSION_PATH': '/org/freedesktop/DisplayManager/Session0', 'XDG_SESSION_TYPE': 'x11', 'LESSCLOSE': '/usr/bin/lesspipe %s %s', 'XDG_DATA_DIRS': '/usr/share/ubuntu:/usr/share/gnome:/usr/local/share:/usr/share:/var/lib/snapd/desktop', 'QT4_IM_MODULE': 'xim', 'QT_IM_MODULE': 'ibus', 'SHLVL': '1', 'COMPIZ_CONFIG_PROFILE': 'ubuntu', 'HOME': '/home/ivan', 'QT_QPA_PLATFORMTHEME': 'appmenu-qt5', 'PS1': '(my_env1) \\[\\e]0;\\u@\\h: \\w\\a\\]${debian_chroot:+($debian_chroot)}\\[\\033\\]\\u@\\h\\[\\033\\]:\\[\\033\\]\\w\\[\\033\\]\\$ ', 'LC_NUMERIC': 'ayc_PE', 'XDG_RUNTIME_DIR': '/run/user/1000', 'PAGER': 'cat', 'IM_CONFIG_PHASE': '1', 'LESSOPEN': '| /usr/bin/lesspipe %s', 'LC_TIME': 'ayc_PE', 'DESKTOP_SESSION': 'ubuntu', 'LANG': 'en_US.UTF-8', 'WINDOWID': '52428810', 'SESSIONTYPE': 'gnome-session', 'LC_IDENTIFICATION': 'ayc_PE', 'GTK_IM_MODULE': 'ibus', 'LIBGL_ALWAYS_

##  Data

In [10]:
N = 100

In [11]:
def build_linear_dataset(N, p=1, noise_std=0.01):
    X = np.random.rand(N, p)
    w = 3 * np.ones(p)
    y = np.matmul(X,w) + np.repeat(1, N) + np.random.normal(0, noise_std, size=N)
    y = y.reshape(N, 1)
    X, y = torch.tensor(X).type(torch.Tensor), torch.tensor(y).type(torch.Tensor)
    data = torch.cat((X,y), 1)
    assert data.shape == (N, p + 1)
    return data
    

In [12]:
class RegressionModel(nn.Module):
    def __init__(self, p):
        super(RegressionModel, self).__init__()
        self.linear = nn.Linear(p, 1)
    
    def forward(self, x):
        return self.linear(x)   

In [13]:
regression_model = RegressionModel(1)

In [14]:
loss_fn = torch.nn.MSELoss(size_average=False)
optim = torch.optim.Adam(regression_model.parameters(), lr=0.05)
num_iterations = 1000 if not smoke_test else 2

In [15]:
def main():
    data = build_linear_dataset(N)
    x_data = data[:, :-1]
    y_data = data[:, -1]
    for j in range(num_iterations):
        y_pred = regression_model(x_data).squeeze(-1)
        loss = loss_fn(y_pred, y_data)
        optim.zero_grad()
        loss.backward()
        optim.step()
        if (j + 1) % 50 == 0:
            print("[iteration %04d] loss: %.4f" % (j + 1, loss.item()))
    print('learned parameters')
    for name, param in regression_model.named_parameters():
        print("%s: %.3f" % (name, param.data.numpy()))

if __name__ == '__main__':
    main()       

[iteration 0050] loss: 14.4159
[iteration 0100] loss: 7.8124
[iteration 0150] loss: 4.0348
[iteration 0200] loss: 1.8165
[iteration 0250] loss: 0.7208
[iteration 0300] loss: 0.2567
[iteration 0350] loss: 0.0858
[iteration 0400] loss: 0.0307
[iteration 0450] loss: 0.0151
[iteration 0500] loss: 0.0113
[iteration 0550] loss: 0.0104
[iteration 0600] loss: 0.0102
[iteration 0650] loss: 0.0102
[iteration 0700] loss: 0.0102
[iteration 0750] loss: 0.0102
[iteration 0800] loss: 0.0102
[iteration 0850] loss: 0.0102
[iteration 0900] loss: 0.0102
[iteration 0950] loss: 0.0102
[iteration 1000] loss: 0.0102
learned parameters
linear.weight: 3.003
linear.bias: 0.998


In [16]:
loc = torch.zeros(1,1)
scale = torch.ones(1,1)
prior = Normal(loc, scale)

In [17]:
loc

tensor([[ 0.]])

In [18]:
scale

tensor([[ 1.]])

In [19]:
prior

Normal()

In [20]:
lifted_module = pyro.random_module('regression_module', regression_model, prior)
sampled_reg_model = lifted_module()

In [21]:
def model(data):
    
    loc, scale = torch.zeros(1, 1), 10 * torch.ones(1, 1)
    bias_loc, bias_scale = torch.zeros(1), 10 * torch.ones(1)
    
    w_prior = Normal(loc, scale).independent(1)
    b_prior = Normal(bias_loc, bias_scale).independent(1)
    
    priors = {'linear.weight': w_prior, 'linear.bias': b_prior}
    
    lifted_module = pyro.random_module("module", regression_model, priors)
    
    lifted_reg_model = lifted_module()
    with pyro.iarange("map", N):
        x_data = data[:, :-1]
        y_data = data[:, -1]
    
        
        prediction_mean = lifted_reg_model(x_data).squeeze(-1)
        
        pyro.sample("obs", 
                    Normal(prediction_mean, 0.1 * torch.ones(data.size(0))),
                    obs=y_data)

In [22]:
softplus = torch.nn.Softplus()

In [23]:
def guide(data):
    # define our variational parameters
    w_loc = torch.randn(1, 1)
    
    # note that we initialize our scales to be pretty narrow
    w_log_sig = torch.tensor(-3.0 * torch.ones(1, 1) + 0.05 * torch.randn(1, 1))
    b_loc = torch.randn(1)
    b_log_sig = torch.tensor(-3.0 * torch.ones(1) + 0.05 * torch.randn(1))
    
    # register learnable params in the param store
    mw_param = pyro.param("guide_mean_weight", w_loc)
    sw_param = softplus(pyro.param("guide_log_scale_weight", w_log_sig))
    mb_param = pyro.param("guide_mean_bias", b_loc)
    sb_param = softplus(pyro.param("guide_log_scale_bias", b_log_sig))
    
    # guide distributions for w and b
    w_dist = Normal(mw_param, sw_param).independent(1)
    b_dist = Normal(mb_param, sb_param).independent(1)
    dists = {'linear.weight': w_dist, 'linear.bias': b_dist}
    
    # overload the parameters in the module with random samples 
    # from the guide distributions
    lifted_module = pyro.random_module("module", regression_model, dists)
    
    # sample a regressor (which also samples w and b)
    return lifted_module()

In [24]:
optim = Adam({"lr": 0.05})
svi = SVI(model, guide, optim, loss=Trace_ELBO())

In [25]:
def main():
    pyro.clear_param_store()
    data = build_linear_dataset(N)
    for j in range(num_iterations):
        
        # calculate the loss and take a gradient step
        loss = svi.step(data)
        if j % 100 == 0:
            print("[iteration %04d] loss: %.4f" % (j + 1, loss / float(N)))
            
if __name__ == '__main__':
    main()

[iteration 0001] loss: 737.4688
[iteration 0101] loss: -0.7664
[iteration 0201] loss: -0.7842
[iteration 0301] loss: -1.0706
[iteration 0401] loss: -1.0227
[iteration 0501] loss: -1.2106
[iteration 0601] loss: -1.1885
[iteration 0701] loss: -1.2329
[iteration 0801] loss: -1.2337
[iteration 0901] loss: -1.2563


In [26]:
for name in pyro.get_param_store().get_all_param_names():
    print("[%s]: %.3f" % (name, pyro.param(name).data.numpy()))


[guide_log_scale_bias]: -3.917
[guide_mean_weight]: 3.009
[guide_log_scale_weight]: -3.800
[guide_mean_bias]: 1.001


In [27]:
X = np.linspace(6, 7, num=20)
y = 3 * X + 1
X, y = X.reshape((20, 1)), y.reshape((20, 1))
x_data, y_data = torch.tensor(X).type(torch.Tensor), torch.tensor(y).type(torch.Tensor)
loss = nn.MSELoss()
y_preds = torch.zeros(20, 1)
for i in range(20):
    # guide does not require the data
    sampled_reg_model = guide(None)
    # run the regression model and add prediction to total
    y_preds = y_preds + sampled_reg_model(x_data)
# take the average of the predictions
y_preds = y_preds / 20
print ("Loss: ", loss(y_preds, y_data).item())


Loss:  0.009167896583676338
